In [8]:
import nest_asyncio
import pandas as pd
import re
from fuzzywuzzy import process
from aiogram import Bot, Dispatcher, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton
from aiogram.filters import Command
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
from aiogram.fsm.context import FSMContext

In [5]:
pip install fuzzywuzzy

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install aiogram

     -------------------------------------- 612.8/612.8 kB 3.0 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
     -------------------------------------- 431.7/431.7 kB 9.0 MB/s eta 0:00:00
     -------------------------------------- 442.4/442.4 kB 6.9 MB/s eta 0:00:00
     -------------------------------------- 166.4/166.4 kB 9.8 MB/s eta 0:00:00
  Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Using cached magic_filter-1.0.12-py3-none-any.whl (11 kB)
  Using cached yarl-1.18.3-cp39-cp39-win_amd64.whl (90 kB)
     ---------------------------------------- 51.8/51.8 kB 2.6 MB/s eta 0:00:00
  Using cached propcache-0.2.1-cp39-cp39-win_amd64.whl (44 kB)
     ---------------------------------------- 2.0/2.0 MB 5.7 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successful

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [9]:
file_path = 'dopuski (1).xlsx'
data = pd.read_excel(file_path)
data.columns = [col.lower() for col in data.columns]
data['yearstart'] = data['yearstart'].astype(str)
data['yearend'] = data['yearend'].astype(str)

# Проверим результат преобразования
print(data[['yearstart', 'yearend']]) 

        yearstart     yearend
0      1981-11-01  1987-12-01
1      2007-07-01  2010-06-01
2      2007-07-01  2010-06-01
3      2008-01-01         nan
4      2008-01-01         nan
...           ...         ...
34709  2010-01-01         nan
34710  1967-04-01  1994-09-01
34711  2013-12-01         nan
34712  2017-06-01         nan
34713  2017-06-01         nan

[34714 rows x 2 columns]


In [10]:
data['makename'].unique()

array(['ABARTH', 'AC', 'ACURA', 'AIXAM', 'ALFA ROMEO', 'ALPINA', 'ALPINE',
       'ARO', 'ASIA MOTORS', 'ASTON MARTIN', 'ASTRA', 'AUDI',
       'AUDI (FAW)', 'AUSTIN', 'AUTOBIANCHI', 'BEDFORD', 'BENTLEY', 'BMC',
       'BMW', 'BMW (BRILLIANCE)', 'BRISTOL', 'BUGATTI', 'BUICK', 'BYD',
       'CADILLAC', 'CARBODIES', 'CATERPILLAR', 'CHERY', 'CHEVROLET',
       'CHEVROLET (SGM)', 'CHRYSLER', 'CITROËN', 'CUPRA', 'DACIA',
       'DAEWOO', 'DAIHATSU', 'DAIMLER', 'DATSUN', 'DE LOREAN',
       'DE TOMASO', 'DODGE', 'DR', 'DS', 'EICHER', 'ERF', 'EUNOS',
       'FERRARI', 'FIAT', 'FISKER', 'FORD', 'FORD (CHANGAN)',
       'FORD ASIA & OCEANIA', 'FORD AUSTRALIA', 'FORD USA', 'FOTON',
       'FPV', 'FREIGHTLINER', 'FSO', 'FUSO (MITSUBISHI)', 'GAZ', 'GEELY',
       'GENESIS', 'GEO', 'GINETTA', 'GMC', 'GREAT WALL', 'HAVAL',
       'HINDUSTAN', 'HINO', 'HOLDEN', 'HONDA', 'HSV', 'HUMMER', 'HYUNDAI',
       'HYUNDAI (BEIJING)', 'INFINITI', 'INNOCENTI', 'INTERNATIONAL',
       'IRAN KHODRO', 'ISUZU', 'IV

In [11]:
data['carname'].unique()

array(['125 TC 2.0 (138.A_)', '1.4 (199.AXN1B)',
       '1.4 ESSEESSE / SUPERSPORT (199.AXX1B)', ..., '1.2 (110557)',
       '1.5 (TF69Y0)', '1.6 16V (TF69YO)'], dtype=object)

In [12]:
#проверим наличие пробелов и других символов
data['yearstart_clean'] = data['yearstart'].str.strip()  #убираем пробелы
data['yearend_clean'] = data['yearend'].str.strip()  #убираем пробелы

print(data[['yearstart', 'yearend', 'yearstart_clean', 'yearend_clean']].head())

    yearstart     yearend yearstart_clean yearend_clean
0  1981-11-01  1987-12-01      1981-11-01    1987-12-01
1  2007-07-01  2010-06-01      2007-07-01    2010-06-01
2  2007-07-01  2010-06-01      2007-07-01    2010-06-01
3  2008-01-01         nan      2008-01-01           nan
4  2008-01-01         nan      2008-01-01           nan


In [ ]:
import nest_asyncio
import pandas as pd
import re
import requests
import time
from fuzzywuzzy import process
from aiogram import Bot, Dispatcher, types, Router
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton
from aiogram.filters import Command
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
import datetime
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup

nest_asyncio.apply()

TOKEN = "7866144691:AAEucBBF9YIeZBJHdCAFDHHUriz-BBFmTqo"
bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher()
router = Router()

data = pd.read_excel('dopuski (1).xlsx')
data.columns = [col.lower() for col in data.columns]
data['yearstart'] = pd.to_datetime(data['yearstart'], errors='coerce').dt.year
data['yearend'] = pd.to_datetime(data['yearend'], errors='coerce').dt.year
data['yearend'] = data['yearend'].fillna(datetime.datetime.now().year).astype(int)

brand_dict = {
    'ABARTH': ['АБАРТ', 'ABARH', 'ABART'],
    'AC': ['АК', 'A-C'],
    'ACURA': ['АКУРА', 'AKURA', 'ACORA'],
    'AIXAM': ['АЙКСАМ', 'AXIAM', 'AIXM'],
    'ALFA ROMEO': ['АЛЬФА РОМЕО', 'ALPHA ROMEO', 'ALFA ROMEU'],
    'ALPINA': ['АЛЬПИНА', 'ALPENA'],
    'ALPINE': ['АЛЬПАЙН', 'ALPIME'],
    'ARO': ['АРО'],
    'ASIA MOTORS': ['АЗИЯ МОТОРС', 'ASYA MOTORS'],
    'ASTON MARTIN': ['АСТОН МАРТИН', 'ASTON MART'],
    'ASTRA': ['АСТРА'],
    'AUDI': ['АУДИ', 'AUDY'],
    'AUDI (FAW)': ['АУДИ (FAW)', 'AUDY (FAW)'],
    'AUSTIN': ['ОСТИН', 'AUSTYN'],
    'AUTOBIANCHI': ['АВТОБЬЯНКИ', 'AUTO BIANKI'],
    'BEDFORD': ['БЕДФОРД', 'BETFORD'],
    'BENTLEY': ['БЕНТЛИ', 'BENTLI', 'BENTLY'],
    'BMC': ['БМЦ', 'B-MC'],
    'BMW': ['БМВ', 'B-MW'],
    'BMW (BRILLIANCE)': ['БМВ (BRILLIANCE)', 'B-MW (BRILLIANCE)'],
    'BRISTOL': ['БРИСТОЛЬ', 'BRYSTOL'],
    'BUGATTI': ['БУГАТТИ', 'BUGATY'],
    'BUICK': ['БЬЮИК', 'BUYK'],
    'BYD': ['БИД', 'B-YD'],
    'CADILLAC': ['КАДИЛЛАК', 'CADILAK'],
    'CARBODIES': ['КАРБОДИС', 'CAR BODY'],
    'CATERPILLAR': ['КАТЕРПИЛЛАР', 'CATERPILAR'],
    'CHERY': ['ЧЕРИ', 'CHERI'],
    'CHEVROLET': ['ШЕВРОЛЕ', 'CHEVROLETE'],
    'CHEVROLET (SGM)': ['ШЕВРОЛЕ (SGM)', 'CHEVROLETE (SGM)'],
    'CHRYSLER': ['КРАЙСЛЕР', 'CHRISLER'],
    'CITROËN': ['СИТРОЕН', 'ЦИТРОЕН', 'CITROEN', 'CITRONE'],
    'CUPRA': ['КУПРА'],
    'DACIA': ['ДАЧИЯ', 'DACYA'],
    'DAEWOO': ['ДЭУ', 'DAWOO'],
    'DAIHATSU': ['ДАИХАТСУ', 'DAHATSU'],
    'DAIMLER': ['ДАЙМЛЕР', 'DAEMLER'],
    'DATSUN': ['ДАТСУН', 'DATSAN'],
    'DE LOREAN': ['ДЕЛОРЕАН', 'DELOREAN'],
    'DE TOMASO': ['ДЕ ТОМАСО', 'DETHOMASO'],
    'DODGE': ['ДОДЖ', 'DOJDE', 'DODJE'],
    'DR': ['ДР'],
    'DS': ['ДС'],
    'EICHER': ['АЙЧЕР', 'EICHAR'],
    'ERF': ['ЕРФ'],
    'EUNOS': ['ЭУНОС', 'E-UNOS'],
    'FERRARI': ['ФЕРРАРИ', 'FERRARY'],
    'FIAT': ['ФИАТ', 'FIYAT'],
    'FISKER': ['ФИСКЕР', 'FISKER'],
    'FORD': ['ФОРД', 'FORDE'],
    'FORD (CHANGAN)': ['ФОРД (CHANGAN)', 'FORDE (CHANGAN)'],
    'FOTON': ['ФОТОН', 'FOTONE'],
    'FREIGHTLINER': ['ФРЕЙТЛАЙНЕР', 'FREITLINER'],
    'FSO': ['ФСО'],
    'FUSO (MITSUBISHI)': ['ФУСО (МИЦУБИСИ)', 'FUSO (MITSUBISHI)'],
    'GAZ': ['ГАЗ'],
    'GEELY': ['ДЖИЛИ', 'GILI'],
    'GENESIS': ['ГЕНЕЗИС', 'GENEZIS'],
    'GEO': ['ГЕО'],
    'GINETTA': ['ДЖИНЕТТА', 'GINETA'],
    'GMC': ['ДЖИ ЭМ СИ', 'G-MC'],
    'GREAT WALL': ['ГРЕЙТ ВОЛЛ', 'GREATWALL'],
    'HAVAL': ['ХАВАЛ', 'HAVALE'],
    'HINO': ['ХИНО', 'HINO'],
    'HONDA': ['ХОНДА', 'HONDE'],
    'HUMMER': ['ХАММЕР', 'HUMER'],
    'HYUNDAI': ['ХЕНДАЙ', 'ХЮНДАЙ', 'HUNDAY', 'HUNDAI', 'HYUNDAY', 'HIUNDAY'],
    'INFINITI': ['ИНФИНИТИ', 'INFINITY'],
    'ISUZU': ['ИСУЗУ', 'ISUSU'],
    'IVECO': ['ИВЕКО', 'IVEKCO'],
    'JAC': ['ДЖАК', 'JAK'],
    'JAGUAR': ['ЯГУАР', 'JAGUARE'],
    'JEEP': ['ДЖИП', 'JEEPE'],
    'KIA': ['КИА', 'KYA'],
    'LADA': ['ЛАДА'],
    'LAMBORGHINI': ['ЛАМБОРГИНИ', 'LAMBORGINI'],
    'LAND ROVER': ['ЛЭНД РОВЕР', 'LANDROVER'],
    'LEXUS': ['ЛЕКСУС', 'LEKUS'],
    'MAZDA': ['МАЗДА', 'MAZDAH'],
    'MCLAREN': ['МАКЛАРЕН', 'MCLARAN'],
    'MERCEDES-BENZ': ['МЕРСЕДЕС', 'МЕРСЕДЕС-БЕНЦ', 'МЕРСЕДЕС БЕНЦ', 'MERCEDES', 'MERCEDES BENZ', 'MERCEDES-BEN'],
    'MITSUBISHI': ['МИЦУБИСИ', 'МИТЦУБИСИ', 'МИТСУБИШИ''MITSUBISI', 'MITZUBISHI', 'MITSUBISHI'],
    'MORGAN': ['МОРГАН', 'MORGAN', 'MORGON', 'MORGAN'],
    'NISSAN': ['НИССАН', 'NISAN'],
    'OPEL': ['ОПЕЛЬ', 'OPELLE'],
    'PEUGEOT': ['ПЕЖО', 'PEJOT'],
    'PORSCHE': ['ПОРШЕ', 'PORSHE'],
    'RENAULT': ['РЕНО', 'RENAUL'],
    'ROLLS-ROYCE': ['РОЛЛС-РОЙС', 'ROLLS ROYS'],
    'SKODA': ['ШКОДА', 'SCODA'],
    'SUBARU': ['СУБАРУ', 'SUBARO'],
    'SUZUKI': ['СУЗУКИ', 'SUZUKY'],
    'TOYOTA': ['ТОЙОТА', 'TOYOTA'],
    'VOLVO': ['ВОЛЬВО', 'VOLVO'],
    'ZAZ': ['ZAZ', 'Заз'],
    'VW': ['ФОЛЬКСВАГЕН', 'VOLKSWAGEN', 'VAG']
}
brand_list = list(brand_dict.keys()) + [item for sublist in brand_dict.values() for item in sublist] #cписок всех возможных вариантов написания брендов.

engine_dict = {
    "бензин": "Бензиновый двигатель", "бензиновый": "Бензиновый двигатель", "gasoline": "Бензиновый двигатель",
    "дизель": "Дизель", "дизельный": "Дизель", "diesel": "Дизель",
    "гибрид": "Полный гибридный привод", "гибридный": "Полный гибридный привод", "hybrid": "Полный гибридный привод",
    "hev": "Полный гибридный привод",
    "мягкий гибрид": "Мягкий гибридный привод", "mhev": "Мягкий гибридный привод",
    "плагин-гибрид": "Гибридный привод с подзарядкой", "phev": "Гибридный привод с подзарядкой",
    "электро": "Электродвигатель", "ev": "Электродвигатель",
    "ванкель": "Двигатель Ванкеля", "wankel": "Двигатель Ванкеля",
    "рекс": "Устройство увеличения запаса хода", "range extender": "Устройство увеличения запаса хода"
}

#класс, который определяет состояния бота. Бот может находиться в одном из трех состояний: ожидание марки, ожидание модели, ожидание комплектации.
class UserInput(StatesGroup):
    waiting_for_brand = State()
    waiting_for_model = State()
    waiting_for_carname = State()
    
selected_brand = None
selected_model = None

import requests
import random


def ask_tinyllama(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "tinyllama",
        "prompt": prompt,
        "stream": False
    }

    # Массив возможных ответов
    responses = [
        "Кажется, мы немного отвлеклись. Давайте вернемся к главному! Пожалуйста, укажите марку, тип двигателя и год выпуска вашего автомобиля, чтобы я мог помочь вам лучше.",
        "Ой, похоже, я не совсем понял ваш запрос. Не могли бы вы уточнить марку, тип двигателя и год выпуска автомобиля? Это поможет мне дать вам точный ответ!",
        "Давайте начнем с самого важного! Пожалуйста, расскажите мне о марке, типе двигателя и годе выпуска вашего автомобиля. Я с радостью помогу!",
        "Чтобы я мог лучше понять вашу ситуацию, мне нужна небольшая информация. Пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля. Спасибо!",
        "Кажется, я не смог разобрать введенные данные. Не волнуйтесь! Пожалуйста, уточните марку, тип двигателя и год выпуска автомобиля, и я обязательно помогу.",
        "Давайте уточним детали! Мне нужна марка, тип двигателя и год выпуска вашего автомобиля. Как только вы поделитесь этой информацией, я смогу предложить вам лучшее решение.",
        "Пожалуйста, расскажите мне немного больше о вашем автомобиле. Укажите марку, тип двигателя и год выпуска, чтобы я мог помочь вам максимально эффективно.",
        "Кажется, я не получил всю необходимую информацию. Не могли бы вы уточнить марку, тип двигателя и год выпуска автомобиля? Это очень важно для решения вашего вопроса!"
    ]

    # Приветствия и общие вопросы
    if "привет" in prompt.lower() or "здравствуй" in prompt.lower() or "здравствуйте" in prompt.lower() or "хай" in prompt.lower():
        return "Привет! Рад встречи с тобой! Пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля, чтобы я мог помочь вам."
    elif "как дела" in prompt.lower():
        return "У меня всё супер! Спасибо, что спросили. Теперь, пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля."
    elif "а зачем" in prompt.lower() or "зачем" in prompt.lower():
        return "Укажите марку, тип двигателя и год выпуска автомобиля, чтобы я смог подобрать тебе подходящее масло!"
    elif "это для чего" in prompt.lower():
        return "Укажите марку, тип двигателя и год выпуска автомобиля, чтобы я смог подобрать тебе подходящее масло!"

    try:
        response = requests.post(url, json=payload, timeout=5)
        response.raise_for_status()
        # Получаем ответ от модели
        model_response = response.json().get("response", "")

        # Если модель вернула пустой или некорректный ответ, используем случайный ответ из массива
        if not model_response.strip():
            return random.choice(responses)
        else:
            return model_response
    except requests.RequestException:
        # В случае ошибки возвращаем случайный ответ из массива
        return random.choice(responses)


#Функция, которая корректирует введенную пользователем марку автомобиля, используя нечеткое сравнение
def correct_brand(brand):
    best_match, score = process.extractOne(brand.upper(), brand_list)
    if score > 80:
        for correct, aliases in brand_dict.items():
            if best_match in aliases or best_match == correct:
                return correct
    return brand.upper()

# Функция обработки типа двигателя
def correct_engine_type(engine):
    return engine_dict.get(engine.lower(), None)

#Функция, которая извлекает год выпуска автомобиля из текста с помощью регулярного выражения
def extract_year(text):
    match = re.search(r'\b(19\d{2}|20\d{2})\b', text)
    return int(match.group()) if match else None

#Функция, которая анализирует введенный пользователем текст и извлекает марку, тип двигателя и год выпуска автомобиля.
def parse_user_input(text):
    # Приводим текст к нижнему регистру для удобства обработки
    text_lower = text.lower()

    # Извлечение года с помощью регулярного выражения
    year_match = re.search(r'\b(19\d{2}|20\d{2})\b', text_lower)
    year = int(year_match.group()) if year_match else None

    # Извлечение типа двигателя
    engine = None
    for engine_keyword, engine_type in engine_dict.items():
        if engine_keyword in text_lower:
            engine = engine_type
            break

    # Если тип двигателя не найден, попробуем найти его в тексте вручную
    if not engine:
        if "дизеле" in text_lower:
            engine = "Дизель"
        elif "бензине" in text_lower or "бензиновом" in text_lower:
            engine = "Бензиновый двигатель"
        elif "гибрид" in text_lower:
            engine = "Полный гибридный привод"
        elif "электро" in text_lower or "электродвигатель" in text_lower:
            engine = "Электродвигатель"

    # Извлечение марки автомобиля
    brand = None
    for word in text_lower.split():
        corrected_brand = correct_brand(word)
        if corrected_brand in brand_dict:
            brand = corrected_brand
            break

    return brand, engine, year

@dp.message(Command("start"))
async def send_welcome(message: types.Message, state: FSMContext):
    await state.clear()
    await message.answer("Привет! Меня зовут VMPботик, и я помогу тебе с подбором масла! Введи, пожалуйста, марку, тип двигателя и год выпуска в любом порядке.")
    await state.set_state(UserInput.waiting_for_brand)

# Обработчик для состояния waiting_for_brand
@dp.message(UserInput.waiting_for_brand)
async def process_input(message: types.Message, state: FSMContext):
    print(f"DEBUG: process_input вызван! Время: {time.time()}, Сообщение: {message.text}")
    if "привет" in message.text.lower() or "здравствуй" in message.text.lower() or "хай" in message.text.lower():
        await message.answer("Привет! Рад встречи с тобой! Пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля, чтобы я мог помочь вам.")
        return
    elif "как дела" in message.text.lower() or "как у тебя дела" in message.text.lower():
        await message.answer("У меня всё супер! Спасибо, что спросили. Теперь, пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля.")
        return
    elif "а зачем" in message.text.lower() or "это для чего" in message.text.lower():
        await message.answer("У меня всё супер! Спасибо, что спросили. Теперь, пожалуйста, укажите марку, тип двигателя и год выпуска автомобиля.")
        return
    # Получаем уже введенные данные
    user_data = await state.get_data()
    brand = user_data.get("selected_brand")
    engine = user_data.get("engine_input")
    year = user_data.get("year_input")
    
    # Если все данные уже введены, пропускаем обработку
    if brand and engine and year:
        await message.answer("Все данные уже введены.")
        return

    # Парсим входное сообщение
    new_brand, new_engine, new_year = parse_user_input(message.text)

    # Обновляем данные только для тех параметров, которые не были введены
    if new_brand and not brand:
        brand = new_brand
    if new_engine and not engine:
        engine = new_engine
    if new_year and not year:
        year = new_year
    
    # Обновляем состояние
    await state.update_data(selected_brand=brand, engine_input=engine, year_input=year)

    # Проверяем, какие данные не хватает
    missing_data = []
    if not brand:
        missing_data.append("марку автомобиля")
    if not engine:
        missing_data.append("тип двигателя (например, бензин, дизель, гибрид, электро)")
    if not year:
        missing_data.append("год выпуска автомобиля")

    # Если данных не хватает, запрашиваем недостающие
    if missing_data:
        missing_data_str = ', '.join(missing_data)
        await message.answer(f"Пожалуйста, укажите {missing_data_str}.")
        return

    # Если все данные введены, продолжаем обработку
    filtered_data = data[
        (data['makename'].str.upper() == brand) &
        (data['engine_type_ru'].str.strip().str.lower() == engine.lower()) &
        (data['yearstart'] <= year) & (data['yearend'] >= year)
    ]

    if filtered_data.empty:
        # Если данные не найдены, отправляем запрос к модели TinyLlama
        response = ask_tinyllama(f"Пользователь ищет информацию о {brand}, {engine}, {year}. Что это может быть?")
        await message.answer(f"Информация не найдена. {response}")
        return

    models = filtered_data['modelname'].unique()
    keyboard = ReplyKeyboardMarkup(
        keyboard=[[KeyboardButton(text=model)] for model in models[:30]],
        resize_keyboard=True,
        one_time_keyboard=True
    )

    await message.answer(f"Выбран бренд: {brand}, двигатель: {engine}, год: {year}\nВыберите модель:", reply_markup=keyboard)
    await state.set_state(UserInput.waiting_for_model)




# Обработчик для состояния waiting_for_model
@dp.message(UserInput.waiting_for_model)
async def process_model_selection(message: types.Message, state: FSMContext):
    global selected_brand, selected_model
    selected_model = message.text.strip()
    await state.update_data(selected_model=selected_model)

    user_data = await state.get_data()
    brand = user_data.get("selected_brand")
    engine = user_data.get("engine_input")

    filtered_data = data[
        (data['makename'].str.upper() == brand) &
        (data['modelname'].str.lower() == selected_model.lower()) &
        (data['engine_type_ru'].str.strip().str.lower() == engine.lower())
    ]

    carnames = filtered_data['carname'].unique()

    if len(carnames) == 0:
        # Если информация о комплектации не найдена, отправляем запрос к модели tinyllama
        response = ask_tinyllama(f"Пользователь ищет информацию о {brand}, {selected_model}, {engine}. Что это может быть?")
        await message.answer(f"Информация о комплектации автомобиля не найдена. {response}")
    elif len(carnames) == 1:
        await process_carname_selection(message, state, carnames[0])
    else:
        keyboard = ReplyKeyboardMarkup(
            keyboard=[[KeyboardButton(text=car)] for car in carnames[:30]],
            resize_keyboard=True,
            one_time_keyboard=True
        )
        await message.answer("Выберите комплектацию автомобиля:", reply_markup=keyboard)
        await state.set_state(UserInput.waiting_for_carname)

# Обработчик для состояния waiting_for_carname
@dp.message(UserInput.waiting_for_carname)
async def process_carname_selection(message: types.Message, state: FSMContext, carname=None):
    if not carname:
        carname = message.text.strip()

    user_data = await state.get_data()
    brand, selected_model, engine = user_data["selected_brand"], user_data["selected_model"], user_data["engine_input"]

    filtered_data = data[
        (data['makename'].str.upper() == brand) &
        (data['modelname'].str.lower() == selected_model.lower()) &
        (data['carname'].str.lower().str.strip() == carname.lower().strip()) &
        (data['engine_type_ru'].str.strip().str.lower() == engine.lower())
    ]

    oils = filtered_data['vmpoil'].dropna().unique()
    if oils.size > 0:
        await message.answer(f"Рекомендуемое масло:\n" + "\n".join(oils))
    else:
        # Если информация о масле не найдена, отправляем запрос к модели tinyllama
        response = ask_tinyllama(f"Пользователь ищет информацию о масле для {brand}, {selected_model}, {carname}, {engine}. Что это может быть?")
        await message.answer(f"Информация о масле не найдена. {response}")

    # Сбрасываем состояние и снова устанавливаем его в waiting_for_brand
    await state.clear()
    await state.set_state(UserInput.waiting_for_brand)
    await message.answer("Вы можете ввести следующую марку, тип двигателя и год выпуска автомобиля.")

dp.include_router(router)

async def main():
    await dp.start_polling(bot)

await main()

DEBUG: process_input вызван! Время: 1740043188.1532202, Сообщение: привет
DEBUG: process_input вызван! Время: 1740043193.6701205, Сообщение: кадилак
DEBUG: process_input вызван! Время: 1740043197.7862854, Сообщение: бензин
DEBUG: process_input вызван! Время: 1740043199.5808532, Сообщение: 2020


Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 7866144691)


DEBUG: process_input вызван! Время: 1740047130.5581079, Сообщение: как дела
DEBUG: process_input вызван! Время: 1740047135.0127609, Сообщение: кадилак
DEBUG: process_input вызван! Время: 1740047138.4351716, Сообщение: бензин
DEBUG: process_input вызван! Время: 1740047140.034281, Сообщение: 2020
DEBUG: process_input вызван! Время: 1740048035.4580958, Сообщение: хавал   бензин
DEBUG: process_input вызван! Время: 1740048043.555775, Сообщение: 22
DEBUG: process_input вызван! Время: 1740048109.1634216, Сообщение: 2025
DEBUG: process_input вызван! Время: 1740048131.5395546, Сообщение: хавал  бензин 2026
DEBUG: process_input вызван! Время: 1740048164.3793788, Сообщение: хавал  бензин
DEBUG: process_input вызван! Время: 1740048212.3182836, Сообщение: хавал бензин 2025


Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 7866144691)


DEBUG: process_input вызван! Время: 1740048300.2308762, Сообщение: хавал бензин
DEBUG: process_input вызван! Время: 1740048300.2578764, Сообщение: эээ алёёёёё
DEBUG: process_input вызван! Время: 1740048307.6366832, Сообщение: привет
DEBUG: process_input вызван! Время: 1740048328.601652, Сообщение: 2026
DEBUG: process_input вызван! Время: 1740048365.2332213, Сообщение: хавал
DEBUG: process_input вызван! Время: 1740048392.733339, Сообщение: Хавал бензин 2026
DEBUG: process_input вызван! Время: 1740048417.5545886, Сообщение: Хавал бензин 2025
DEBUG: process_input вызван! Время: 1740048429.3434434, Сообщение: Да
DEBUG: process_input вызван! Время: 1740048450.873482, Сообщение: хавал бензин
DEBUG: process_input вызван! Время: 1740048453.261442, Сообщение: 2025
DEBUG: process_input вызван! Время: 1740048548.8781285, Сообщение: хавал бензин 2022


Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 7866144691)


DEBUG: process_input вызван! Время: 1740049046.4281855, Сообщение: привет
DEBUG: process_input вызван! Время: 1740049050.6508641, Сообщение: как дела
DEBUG: process_input вызван! Время: 1740049062.094533, Сообщение: кадилак
DEBUG: process_input вызван! Время: 1740049065.3093088, Сообщение: бензин
DEBUG: process_input вызван! Время: 1740049066.4431465, Сообщение: 2020
DEBUG: process_input вызван! Время: 1740049377.0782974, Сообщение: привет
DEBUG: process_input вызван! Время: 1740049384.1885731, Сообщение: как дела
DEBUG: process_input вызван! Время: 1740049396.8557012, Сообщение: зачем
DEBUG: process_input вызван! Время: 1740049444.532081, Сообщение: бензин
DEBUG: process_input вызван! Время: 1740049455.3445718, Сообщение: кадилак
DEBUG: process_input вызван! Время: 1740049460.0976384, Сообщение: 2019
DEBUG: process_input вызван! Время: 1740050372.7200756, Сообщение: привет
DEBUG: process_input вызван! Время: 1740050376.5402427, Сообщение: как дела
DEBUG: process_input вызван! Время: 1

Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 7866144691)
Failed to fetch updates - TelegramNetworkError: HTTP Client says - ClientConnectorDNSError: Cannot connect to host api.telegram.org:443 ssl:default [getaddrinfo failed]
Sleep for 1.339684 seconds and try again... (tryings = 1, bot id = 7866144691)
Failed to fetch updates - TelegramNetworkError: HTTP Client says - ClientConnectorDNSError: Cannot connect to host api.telegram.org:443 ssl:default [getaddrinfo failed]
Sleep for 1.728780 seconds and try again... (tryings = 2, bot id = 7866144691)
Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
Sleep for 2.466971 seconds and try again... (tryings = 3, bot id = 7866144691)
Failed to fetch updates - TelegramNetworkError: HTTP Client says - ClientConnectorDNSError: Cannot connect to host api.telegram.org:443 ssl:default [getaddrinfo failed]